In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.backend import sigmoid
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import optimizers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.layers import Conv1D, Dense, Dropout, LayerNormalization, Bidirectional, LSTM, GRU, Layer, SpatialDropout1D, GlobalAveragePooling1D
from tensorflow.keras.layers import Lambda, Reshape, Flatten, Input, MultiHeadAttention, Flatten, Concatenate, Add
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import pickle
import seaborn

2025-11-30 00:34:39.974285: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def clean_and_interpolate(data):
    df = pd.DataFrame(data).astype(np.float32)
    
    df_linear = df.interpolate(method='linear', limit_direction='both', axis=1, limit=None)
    df_spline = df_linear.interpolate(method='spline', order=3, axis=1).fillna(method='ffill', axis=1).fillna(method='bfill', axis=1)

    df_final = df_spline.fillna(0)
    
    # Clip extreme values
    df_interpolated_fin = df_final.clip(-1e6, 1e6)
    print(f"Final NaN count: {df_interpolated_fin.isnull().sum().sum()}")
    result = df_interpolated_fin.values
    
    return result

In [3]:
# Slope + elevation - fixed constraint

def multi_focal_loss_slope_elevation_constraint(slope_values, elevation_values,
                                                slope_threshold=40, elevation_threshold=2000,
                                                alpha=0.25, gamma=2.0,
                                                lambda_slope=0.4, lambda_elevation=0.3):
    def loss(y_true, y_pred):
        epsilon = tf.keras.backend.epsilon()
        y_pred_clipped = tf.clip_by_value(y_pred, epsilon, 1.0 - epsilon)
        ce = -y_true * tf.math.log(y_pred_clipped)
        p_t = tf.where(tf.equal(y_true, 1), y_pred_clipped, 1 - y_pred_clipped)
        
        # Alpha weighting for class imbalance
        alpha_t = tf.where(tf.equal(y_true[:, 1], 1), alpha, 1 - alpha)
        
        # Focal loss calculation
        focal_weight = tf.expand_dims(alpha_t, 1) * tf.pow((1 - p_t), gamma)
        focal_loss = focal_weight * ce
        focal_loss = tf.reduce_mean(tf.reduce_sum(focal_loss, axis=1))
        
        # Get batch data
        batch_size = tf.shape(y_pred)[0]
        slope_batch = tf.gather(slope_values, tf.range(batch_size))
        elevation_batch = tf.gather(elevation_values, tf.range(batch_size))
        
        # ISA predictions
        isa_pred = y_pred[:, 0]
        
        # CONSTRAINT 1: Slope constraint
        slope_mask = tf.cast(slope_batch > slope_threshold, tf.float32)
        slope_penalty = tf.reduce_mean(tf.multiply(slope_mask, isa_pred))
        
        # CONSTRAINT 2: Elevation constraint
        elevation_mask = tf.cast(elevation_batch > elevation_threshold, tf.float32)
        elevation_penalty = tf.reduce_mean(tf.multiply(elevation_mask, isa_pred))
        
        # Combine all losses
        total_loss = focal_loss + (lambda_slope * slope_penalty) + (lambda_elevation * elevation_penalty)
        
        return total_loss
    
    return loss

In [6]:
def prepare_multi_region_data(citarum_01_paths, citarum_02_paths, citarum_03_paths, jkt_paths):
    def load_region_data(paths):
        # Load all dataframes for a region
        ndvi_df = pd.read_csv(paths['ndvi'],delimiter=';', encoding='utf-8-sig',decimal=',')
        mndwi_df = pd.read_csv(paths['mndwi'],delimiter=';', encoding='utf-8-sig',decimal=',')
        ndbi_df = pd.read_csv(paths['ndbi'],delimiter=';', encoding='utf-8-sig',decimal=',')
        ndbsi_df = pd.read_csv(paths['ndbsi'],delimiter=';', encoding='utf-8-sig',decimal=',')
        cbi_df = pd.read_csv(paths['cbi'],delimiter=';', encoding='utf-8-sig',decimal=',')
        uci_df = pd.read_csv(paths['uci'],delimiter=';', encoding='utf-8-sig',decimal=',')
        print('NDVI shape before int:', ndvi_df.shape)
        
        # Extract features
        ndvi_features = ndvi_df.iloc[:,5:29].values
        mndwi_features = mndwi_df.iloc[:,5:29].values
        ndbi_features = ndbi_df.iloc[:,5:29].values
        ndbsi_features = ndbsi_df.iloc[:,5:29].values
        cbi_features = cbi_df.iloc[:,5:29].values
        uci_features = uci_df.iloc[:,5:29].values
        print('NDVI shape after int:', ndvi_features.shape)
        
        # Clean and interpolate
        ndvi_features = clean_and_interpolate(ndvi_features)
        mndwi_features = clean_and_interpolate(mndwi_features)
        ndbi_features = clean_and_interpolate(ndbi_features)
        ndbsi_features = clean_and_interpolate(ndbsi_features)
        cbi_features = clean_and_interpolate(cbi_features)
        uci_features = clean_and_interpolate(uci_features)
        
        # Combine features
        X = np.concatenate([ndvi_features, mndwi_features, ndbi_features, 
                            ndbsi_features, cbi_features, uci_features], axis=1)

        print("X afer concatenate", X.shape)
        
        # Reshape
        X = X.reshape(X.shape[0], 24, 6)
        print("X afer reshape", X.shape)
        
        # Get labels, slope and aspect
        labels = ndvi_df.iloc[:, 1].values
        slope_values = ndvi_df.iloc[:,29].values
        aspect_values = ndvi_df.iloc[:,30].values
        
        # return X, labels, slope_values
        return X, labels, slope_values, aspect_values

    # Load data for both regions
    X_citarum_01, labels_citarum_01, slope_citarum_01, elev_citarum_01 = load_region_data(citarum_01_paths)
    X_citarum_02, labels_citarum_02, slope_citarum_02, elev_citarum_02 = load_region_data(citarum_02_paths)
    X_citarum_03, labels_citarum_03, slope_citarum_03, elev_citarum_03 = load_region_data(citarum_03_paths)
    X_jkt, labels_jkt, slope_jkt, elev_jkt = load_region_data(jkt_paths)
    
    # Combine data from both regions
    X_combined = np.concatenate([X_citarum_01, X_citarum_02, X_citarum_03, X_jkt], axis=0)
    labels_combined = np.concatenate([labels_citarum_01, labels_citarum_02, labels_citarum_03, labels_jkt])
    slope_combined = np.concatenate([slope_citarum_01, slope_citarum_02, slope_citarum_03, slope_jkt])
    elev_combined = np.concatenate([elev_citarum_01, elev_citarum_02, elev_citarum_03, elev_jkt])
    
    # Compute class weights on combined data
    unique_classes = np.unique(labels_combined)

    print("Labels combined shape:", labels_combined.shape)
    print("Labels combined type:", type(labels_combined[0]) if len(labels_combined) > 0 else "Empty")
    print("Unique classes from np.unique:", unique_classes)
    print("Unique classes type:", type(unique_classes[0]) if len(unique_classes) > 0 else "Empty")
    print("All unique values in labels_combined:", set(labels_combined))
    print("Any NaN values?", np.any(pd.isna(labels_combined)))

    # Check if there are any labels in y that aren't in classes
    missing_labels = set(labels_combined) - set(unique_classes)
    print("Labels in y but not in classes:", missing_labels)
    
    class_weights = compute_class_weight(
        class_weight='balanced',
        classes=unique_classes,
        y=labels_combined)
    
    class_weight_dict = dict(enumerate(class_weights))
    print("Class weights:", class_weight_dict)
    
    y = np.asarray(labels_combined)
    print(y)
    print(len(np.unique(y)))
    
    # Encode labels
    label_encoder = LabelEncoder()
    y_combined = label_encoder.fit_transform(labels_combined)
    y_combined = to_categorical(y_combined)
    print(y_combined)
    
    # Split combined data
    X_train, X_test, y_train, y_test, slope_train, slope_test, elev_train, elev_test = train_test_split(X_combined, y_combined,
                                                                                                            slope_combined, elev_combined,
                                                                                                            test_size=0.3,random_state=42,
                                                                                                            stratify=y_combined)
    
    # Convert slope values to tensorflow constant
    slope_train = tf.constant(slope_train, dtype=tf.float32)
    slope_test = tf.constant(slope_test, dtype=tf.float32)

    # Convert aspect values to tensorflow constant
    elev_train = tf.constant(elev_train, dtype=tf.float32)
    elev_test = tf.constant(elev_test, dtype=tf.float32)
    
    return (X_train, X_test, y_train, y_test, label_encoder, class_weight_dict, slope_train, slope_test, elev_train, elev_test, y)

In [7]:
citarum_01_paths = {
    'ndvi': '/home/jupyter-bryan/ISA_Data/Sample_Points_ISA_Citarum_NDVI_ROI_01.csv',
    'mndwi': '/home/jupyter-bryan/ISA_Data/Sample_Points_ISA_Citarum_MNDWI_ROI_01.csv',
    'ndbi': '/home/jupyter-bryan/ISA_Data/Sample_Points_ISA_Citarum_NDBI_ROI_01.csv',
    'ndbsi': '/home/jupyter-bryan/ISA_Data/Sample_Points_ISA_Citarum_NDBSI_ROI_01.csv',
    'cbi': '/home/jupyter-bryan/ISA_Data/Sample_Points_ISA_Citarum_CBI_ROI_01.csv',
    'uci': '/home/jupyter-bryan/ISA_Data/Sample_Points_ISA_Citarum_UCI_ROI_01.csv'
}

citarum_02_paths = {
    'ndvi': '/home/jupyter-bryan/ISA_Data/Sample_Points_ISA_Citarum_NDVI_ROI_02.csv',
    'mndwi': '/home/jupyter-bryan/ISA_Data/Sample_Points_ISA_Citarum_MNDWI_ROI_02.csv',
    'ndbi': '/home/jupyter-bryan/ISA_Data/Sample_Points_ISA_Citarum_NDBI_ROI_02.csv',
    'ndbsi': '/home/jupyter-bryan/ISA_Data/Sample_Points_ISA_Citarum_NDBSI_ROI_02.csv',
    'cbi': '/home/jupyter-bryan/ISA_Data/Sample_Points_ISA_Citarum_CBI_ROI_02.csv',
    'uci': '/home/jupyter-bryan/ISA_Data/Sample_Points_ISA_Citarum_UCI_ROI_02.csv'
}

citarum_03_paths = {
    'ndvi': '/home/jupyter-bryan/ISA_Data/Sample_Points_ISA_Citarum_NDVI_ROI_03.csv',
    'mndwi': '/home/jupyter-bryan/ISA_Data/Sample_Points_ISA_Citarum_MNDWI_ROI_03.csv',
    'ndbi': '/home/jupyter-bryan/ISA_Data/Sample_Points_ISA_Citarum_NDBI_ROI_03.csv',
    'ndbsi': '/home/jupyter-bryan/ISA_Data/Sample_Points_ISA_Citarum_NDBSI_ROI_03.csv',
    'cbi': '/home/jupyter-bryan/ISA_Data/Sample_Points_ISA_Citarum_CBI_ROI_03.csv',
    'uci': '/home/jupyter-bryan/ISA_Data/Sample_Points_ISA_Citarum_UCI_ROI_03.csv'
}

jkt_paths = {
    'ndvi': '/home/jupyter-bryan/ISA_Data/Sample_Points_ISA_DKI_NDVI.csv',
    'mndwi': '/home/jupyter-bryan/ISA_Data/Sample_Points_ISA_DKI_MNDWI.csv',
    'ndbi': '/home/jupyter-bryan/ISA_Data/Sample_Points_ISA_DKI_NDBI.csv',
    'ndbsi': '/home/jupyter-bryan/ISA_Data/Sample_Points_ISA_DKI_NDBSI.csv',
    'cbi': '/home/jupyter-bryan/ISA_Data/Sample_Points_ISA_DKI_CBI.csv',
    'uci': '/home/jupyter-bryan/ISA_Data/Sample_Points_ISA_DKI_UCI.csv'
}

# Prepare the combined data
X_train, X_test, y_train, y_test, label_encoder, class_weight_dict, slope_train, slope_test, elev_train, elev_test, y = prepare_multi_region_data(citarum_01_paths, 
                                                                                                                                                      citarum_02_paths, 
                                                                                                                                                      citarum_03_paths,
                                                                                                                                                      jkt_paths)

NDVI shape before int: (2473, 31)
NDVI shape after int: (2473, 24)
Final NaN count: 0


/tmp/ipykernel_606415/3811670575.py:5: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_spline = df_linear.interpolate(method='spline', order=3, axis=1).fillna(method='ffill', axis=1).fillna(method='bfill', axis=1)
/tmp/ipykernel_606415/3811670575.py:5: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_spline = df_linear.interpolate(method='spline', order=3, axis=1).fillna(method='ffill', axis=1).fillna(method='bfill', axis=1)


Final NaN count: 0
Final NaN count: 0


/tmp/ipykernel_606415/3811670575.py:5: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_spline = df_linear.interpolate(method='spline', order=3, axis=1).fillna(method='ffill', axis=1).fillna(method='bfill', axis=1)
/tmp/ipykernel_606415/3811670575.py:5: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_spline = df_linear.interpolate(method='spline', order=3, axis=1).fillna(method='ffill', axis=1).fillna(method='bfill', axis=1)


Final NaN count: 0
Final NaN count: 0


...

/tmp/ipykernel_606415/3811670575.py:5: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_spline = df_linear.interpolate(method='spline', order=3, axis=1).fillna(method='ffill', axis=1).fillna(method='bfill', axis=1)
/tmp/ipykernel_606415/3811670575.py:5: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_spline = df_linear.interpolate(method='spline', order=3, axis=1).fillna(method='ffill', axis=1).fillna(method='bfill', axis=1)
/tmp/ipykernel_606415/3811670575.py:5: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_spline = df_linear.interpolate(method='spline', order=3, axis=1).fillna(method='ffill', axis=1).fillna(method='bfill', axis=1)


Final NaN count: 0
Final NaN count: 0
Final NaN count: 0


/tmp/ipykernel_606415/3811670575.py:5: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_spline = df_linear.interpolate(method='spline', order=3, axis=1).fillna(method='ffill', axis=1).fillna(method='bfill', axis=1)
/tmp/ipykernel_606415/3811670575.py:5: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_spline = df_linear.interpolate(method='spline', order=3, axis=1).fillna(method='ffill', axis=1).fillna(method='bfill', axis=1)


Final NaN count: 0
Final NaN count: 0
X afer concatenate (2205, 144)
X afer reshape (2205, 24, 6)
Labels combined shape: (7183,)
Labels combined type: <class 'numpy.int64'>
Unique classes from np.unique: [1 2 3 4 5]
Unique classes type: <class 'numpy.int64'>
All unique values in labels_combined: {np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5)}
Any NaN values? False
Labels in y but not in classes: set()
Class weights: {0: np.float64(0.4424391746227287), 1: np.float64(0.8934079601990049), 2: np.float64(0.9115482233502538), 3: np.float64(3.1435448577680525), 4: np.float64(4.8698305084745765)}
[1 1 1 ... 2 2 2]
5
[[1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 ...
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]]


/tmp/ipykernel_606415/3811670575.py:5: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_spline = df_linear.interpolate(method='spline', order=3, axis=1).fillna(method='ffill', axis=1).fillna(method='bfill', axis=1)
I0000 00:00:1764434117.196723  606415 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 7775 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3080, pci bus id: 0000:01:00.0, compute capability: 8.6


In [8]:
class MambaLayer(layers.Layer):
    def __init__(self, d_model, d_state=16, **kwargs):
        super().__init__(**kwargs)
        self.d_model = d_model
        self.d_state = d_state
        
    def build(self, input_shape):
        # All projections in one go
        self.delta_proj = layers.Dense(self.d_state, use_bias=False)
        self.B_proj = layers.Dense(self.d_state, use_bias=False) 
        self.C_proj = layers.Dense(self.d_state, use_bias=False)
        
        # State matrix A and skip connection D
        self.A = self.add_weight(shape=(self.d_model, self.d_state), initializer='random_normal', trainable=True)
        self.D = self.add_weight(shape=(self.d_model,), initializer='random_normal', trainable=True)
        
        super().build(input_shape)
    
    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        seq_len = tf.shape(inputs)[1]
        
        # Generate selective parameters
        delta = tf.nn.softplus(self.delta_proj(inputs))  # (batch, seq_len, d_state)
        B = self.B_proj(inputs)  # (batch, seq_len, d_state)
        C = self.C_proj(inputs)  # (batch, seq_len, d_state)
        
        # Initialize state
        h = tf.zeros((batch_size, self.d_model, self.d_state))
        A_matrix = -tf.exp(self.A)  # Stable A matrix
        
        # Pre-allocate output tensor
        outputs = tf.TensorArray(dtype=tf.float32, size=seq_len, dynamic_size=False)
        
        # Selective scan - process sequence step by step
        for t in tf.range(seq_len):
            u_t = inputs[:, t, :]  # Current input
            delta_t = delta[:, t, :]
            B_t = B[:, t, :]
            C_t = C[:, t, :]
            
            # Discretize and update state
            A_discrete = tf.exp(tf.expand_dims(delta_t, 1) * tf.expand_dims(A_matrix, 0))
            h = A_discrete * h + tf.expand_dims(delta_t, 1) * tf.expand_dims(B_t, 1) * tf.expand_dims(u_t, -1)
            
            # Generate output
            y = tf.reduce_sum(tf.expand_dims(C_t, 1) * h, axis=-1) + self.D * u_t
            outputs = outputs.write(t, y)
        
        # Stack all outputs
        return tf.transpose(outputs.stack(), [1, 0, 2])
    
    def get_config(self):
        """Required method for serialization"""
        config = super().get_config()
        config.update({
            "d_model": self.d_model,
            "d_state": self.d_state,
        })
        return config
    
    @classmethod
    def from_config(cls, config):
        """Required method for deserialization"""
        return cls(**config)

In [9]:
n_classes=len(label_encoder.classes_)
n_features=6

def create_mamba_model(n_classes, n_features, n_timesteps=24, d_model=256, n_layers=8):
    input_layer = Input(shape=(n_timesteps, n_features))
    x = Dense(d_model)(input_layer)
    
    # Stack Mamba layers with residual connections
    for i in range(n_layers):
        norm_x = LayerNormalization()(x)
        mamba_out = MambaLayer(d_model, d_state=16)(norm_x)
        x = tf.keras.layers.add([x, mamba_out])
    
    x = Flatten()(x)
    x = Dropout(0.1)(x)
    output_layer = Dense(n_classes, activation='softmax')(x)

    return Model(inputs=input_layer, outputs=output_layer)

model = create_mamba_model(n_classes, n_features)
model.summary()

model.compile(loss=multi_focal_loss_slope_elevation_constraint(
                    slope_train, elev_train,  
                    slope_threshold=40, elevation_threshold=2000,
                    alpha=0.25, gamma=2.0, lambda_slope=0.4, lambda_elevation=0.3),
              optimizer=Adam(learning_rate=0.001), 
              metrics=['accuracy'])

checkpoint = ModelCheckpoint(filepath='/home/jupyter-bryan/ISA_Data/ISA_Orig_Mamba.h5',
                             monitor='val_accuracy',
                             save_best_only=True,
                             verbose=1,
                             mode='max',
                             save_weights_only=True)

early_stop = EarlyStopping(monitor='val_accuracy',
                           patience=100,
                           restore_best_weights=True,
                           mode='max')

reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', 
                              factor=0.1,
                              patience=30, 
                              min_lr=0.00001)

history = model.fit(X_train, y_train, validation_data=(X_test, y_test), 
                    batch_size =20,
                    epochs=500, 
                    callbacks = [early_stop, reduce_lr], 
                    class_weight = class_weight_dict)

model_state = {
    'weights': model.get_weights(),
    'config': {
        'n_classes': n_classes,
        'n_features': n_features, 
        'n_timesteps': 24,
        'd_model': 256,
        'n_layers': 8
    },
    'history': history.history
}

# # Save with pickle
# save_path = 'D:/ISA_Citarum/Saved_Model/ISA_Citarum_Multi_Orig_Mamba.pkl'
# with open(save_path, 'wb') as f:
#     pickle.dump(model_state, f)

# print(f"Model state saved to: {save_path}")

OperatorNotAllowedInGraphError: Exception encountered when calling MambaLayer.call().

[1mCould not automatically infer the output shape / dtype of 'mamba_layer' (of type MambaLayer). Either the `MambaLayer.call()` method is incorrect, or you need to implement the `MambaLayer.compute_output_spec() / compute_output_shape()` method. Error encountered:

Iterating over a symbolic `tf.Tensor` is not allowed. You can attempt the following resolutions to the problem: If you are running in Graph mode, use Eager execution mode or decorate this function with @tf.function. If you are using AutoGraph, you can try decorating this function with @tf.function. If that does not work, then you may be using an unsupported feature or your source code may not be visible to AutoGraph. See https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/autograph/g3doc/reference/limitations.md#access-to-source-code for more information.[0m

Arguments received by MambaLayer.call():
  • args=('<KerasTe

In [ ]:
n_epochs = len(history.history['loss'])

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(n_epochs) #change it based on epoch needed to finish building the model

plt.figure(figsize=(16, 16))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()